#Main file
This main file is based on the scripts that I had for minimizing the machines from Netlogo. The original was made to capture output from Behaviour Space, processs the machines and then print it to use in Stata. This is too cumbersome, so decided to implement and do everything in Python so I can centralise all the analysis and work on the next algorithms such as Joint Machines, frequencies and unused behavioural states in order to analyse properly how the transitions are happening.

#1 Receiving the Netlogo output
The first step is to load the files. Also, as the global variables of interest in order to associate it with the output name file.

In [1]:
import pandas as pd
import numpy as np
import copy
import minimization as minz #My script in same folder for minimization routines

#Choose here the Globals and Name used for the experiment to load.
#Make sure the files exist (i.e. from Netlogo simulations)
experiment_clue = "trial"
n_states = 4
n_signals = 0
n_rounds = 5
N = 40
n_parents = 20

globals_list = (experiment_clue, n_states, n_signals, n_rounds, N, n_parents) #Save them as a list

#Experiment name based on the chosen experiment_clue and globals
chosen_experiment = "%s_states_%s_signal_%s_rounds_%s_N_%s_parents_%s.txt" % globals_list

#Both files have to use the same "chosen experiment" (to make sure come from the same simulation in Netlogo)
summary_file_name = "summary_" + chosen_experiment #Summary output
strategies_file_name = "strategies_" + chosen_experiment #Strategies output

#Path to Netlogo outputs
netlogo_folder = "/Users/luisalejandrolee/Dropbox/Thesis Phd/\
Coordination autos Chapter three/outputs_to_text/" #Netlogo outputs in this folder

#Read files and save them as data
df_sum = pd.read_csv(netlogo_folder + summary_file_name)
df_strat = pd.read_csv(netlogo_folder + strategies_file_name)

#2 Minimise the automata
Use the functions to have a simple code here for minimising the auto and storing other relevant variables (as available states, etc)

In [2]:
#Other required globals
n_obs = 2 if n_signals == 0 else 4 #Define here (or change) the possible observations of the machines

canon_autos_list = [] #Create empty lists to store the processed autos below
min_autos_list = []


for i in df_strat.index: #For each row...
    netlogo_auto = df_strat.auto_long[i] #... for all netlogo_auto
    big_auto = minz.to_format_netlogo_auto(netlogo_auto) #Use function to convert the raw Netlogo auto in a list format
    init_state = big_auto[0] #Save initial state of the machine
    normal_auto = minz.new_empty_auto(n_obs, n_states) #Use function to create a new empty auto as a numpy array

    # Next block it to fill the new 'normal_auto' with the information from big_auto.
    # The objective is that normal_auto=big_auto but as an array (instead of a list)
    my_index = xrange(1, len(big_auto), n_obs + 1) # Each number in the index is where a state starts
    for i, j in enumerate(my_index):
        normal_auto['actions'][i] = big_auto[j]
        normal_auto['transitions'][i] = big_auto[j + 1:j + n_obs + 1]
        
    canon_auto = minz.convert_to_canonical(normal_auto, n_states, init_state, n_obs) #Use function for canonical form
    
    access_states = len(canon_auto) #n_states now is only the accesible states of the machine (before minimization)
    
    #Use function to get minimum behavioural equivalent auto
    #Passes "0" as 3rd argument because that's init_state now (always 0 for canonical auto)
    min_auto = minz.minimized_automaton(canon_auto, access_states, 0, n_obs)
    
    #Update autos lists
    canon_autos_list.append(canon_auto) #Store proccessed autos in the corresponding list
    min_autos_list.append(min_auto)
    
#Add the processed autos lists as columns to df_strat
df_strat["canon_autos"] = canon_autos_list #Add the lists with autos to the dataframe
df_strat["min_autos"] = min_autos_list

#3) Some variables important to keep

##3.1 Accesible states and minimum behavioural states

In [3]:
#Accesible states
df_strat["access_states"] = -1 #Initialise negative to capture errors later
for i in df_strat.index: #For all canonical autos
    states = len(df_strat.canon_autos[i]) #Accesible states is the lenght of the canonical auto
    df_strat.loc[i, "access_states"] = states #Save to dataframe

#Minimum behavioral states
df_strat["min_states"] = -1 #Initialise negative to capture errors later
for i in df_strat.index: #For all canonical autos
    states = len(df_strat.min_autos[i]) #Min states is the lenght of the min auto, taken as proxy for complexity
    df_strat.loc[i, "min_states"] = states #Save to dataframe

In [4]:

#FOR EXPLAINING STUFF

#joint machines (check Java code)
#joint machine frequencies
#other machines frequencies (minimised)
#regime identification (using joint machines)
#Graphs and stats (automate them)

##Next: two things with John: 
#1) unused behavior measure (to test effect in regime changes)
#2) describe graphically his transition explanations



In [5]:
df_strat

,generation,ID,population,score,auto_long,canon_autos,min_autos,access_states,min_states
0,0,47,row,1.265,[0 A 3 0 B 2 1 A 1 0 A 3 2],"[[A, [1, 0]], [A, [1, 2]], [A, [3, 0]], [B, [2...","[[A, [1, 0]], [A, [1, 2]], [A, [3, 0]], [B, [2...",4,4
1,0,39,row,1.245,[1 B 1 3 A 3 2 B 1 2 B 2 0],"[[A, [1, 2]], [B, [2, 3]], [B, [0, 2]], [B, [0...","[[A, [1, 2]], [B, [2, 3]], [B, [0, 2]], [B, [0...",4,4
2,0,73,column,1.845,[0 B 2 2 A 3 0 A 2 2 B 0 3],"[[B, [1, 1]], [A, [1, 1]]]","[[B, [1, 1]], [A, [1, 1]]]",2,2
3,0,71,row,1.435,[3 A 0 2 B 1 2 B 0 1 B 1 1],"[[B, [1, 1]], [B, [1, 2]], [B, [3, 1]], [A, [3...","[[B, [1, 1]], [B, [1, 2]], [B, [3, 1]], [A, [3...",4,4
4,0,63,row,1.185,[2 B 2 2 B 0 1 B 2 2 A 1 2],"[[B, [0, 0]]]","[[B, [0, 0]]]",1,1
5,0,43,column,1.785,[2 A 2 1 A 0 0 A 0 1 B 3 1],"[[A, [1, 2]], [A, [0, 2]], [A, [1, 1]]]","[[A, [0, 0]]]",3,1
6,0,42,column,1.250,[0 A 0 2 B 2 0 B 1 3 B 2 0],"[[A, [0, 1]], [B, [2, 3]], [B, [1, 0]], [B, [1...","[[A, [0, 1]], [B, [2, 2]], [B, [1, 0]]]",4,3
7,0,0,row,1.535,[1 A 0 1 B 0 3 B 2 1 B 1 2],"[[B, [1, 2]], [A, [1, 0]], [B, [0, 3]], [B, [3...","[[B, [1, 2]], [A, [1, 0]], [B, [0, 3]], [B, [3...",4,4
8,0,9,column,1.765,[0 A 2 2 A 1 1 A 3 0 B 2 1],"[[A, [1, 1]], [A, [2, 0]], [B, [1, 3]], [A, [3...","[[A, [1, 1]], [A, [2, 0]], [B, [1, 3]], [A, [3...",4,4
9,0,14,column,1.800,[0 A 2 1 B 0 0 A 2 2 A 3 1],"[[A, [1, 2]], [A, [1, 1]], [B, [0, 0]]]","[[A, [1, 2]], [A, [1, 1]], [B, [0, 0]]]",3,3


In [10]:
#Some trials on how to index the autos from
# the dataframe df_strat(kept for references for my short memory, but delete at any moment)
df_strat.canon_autos[0] #Auto

array([('A', [1, 0]), ('A', [1, 2]), ('A', [3, 0]), ('B', [2, 3])], 
      dtype=[('actions', 'S1'), ('transitions', '<i4', (2,))])

In [23]:
type (df_strat.canon_autos[0])

numpy.ndarray

In [24]:
df_strat.canon_autos[0][0] #First state

('A', [1, 0])

In [26]:
type (df_strat.canon_autos[0][0]) #First state

numpy.void

In [12]:
df_strat.canon_autos[0][0][0] #Action

'A'

In [13]:
df_strat.canon_autos[0][0][1] #Transitions list

array([1, 0], dtype=int32)

In [14]:
df_strat.canon_autos[0][0][1][0]#Transition number

1

In [15]:
df_strat.canon_autos[0]["actions"]

array(['A', 'A', 'A', 'B'], 
      dtype='|S1')

In [27]:
df_strat.min_autos[0]["transitions"]

array([[1, 0],
       [1, 2],
       [3, 0],
       [2, 3]], dtype=int32)